In [5]:
import os
import re
import glob
import pandas as pd

# 0. Create an empty DataFrame with columns "ID", "slot1", and "Description"
df = pd.DataFrame(columns=["ID", "Description", "Notes", "slot1", "slot2"])

# Define the folder containing Excel files
folder_path = "Promos"  # Update this to your actual folder path
excel_files = glob.glob(os.path.join(folder_path, "*.xlsx"))  # Adjust pattern if needed (e.g., *.xls)

# Regular expression pattern to extract the ID (format: 3 digits, a dash, 3 digits)
id_pattern = re.compile(r'(\d{3}-\d{3})')

for file in excel_files:
    # Extract the ID from the filename
    filename = os.path.basename(file)
    id_match = id_pattern.search(filename)
    if not id_match:
        # Skip files that don't match the ID pattern
        continue
    file_id = id_match.group(1)

    # 5. Skip this file if the ID is already in the DataFrame
    if file_id in df["ID"].values:
        continue

    # Read the Excel file without assuming headers
    try:
        data = pd.read_excel(file, header=None)
    except Exception as e:
        print(f"Error reading {file}: {e}")
        continue

    # Find the row where column A equals "Product Name" (case-insensitive, stripping spaces)
    product_name_rows = data[data[0].astype(str).str.strip().str.lower() == "product name"].index
    if not product_name_rows.empty:
        # Get the value from column B (index 1)
        product_desc = data.iloc[product_name_rows[0], 1]
    else:
        product_desc = ""

    # 3. Find the row where column A equals "material" (case-insensitive, stripping spaces)
    material_rows = data[data[0].astype(str).str.strip().str.lower() == "material"].index
    if material_rows.empty:
        # If no "material" row is found, skip this file
        continue
    start_row = material_rows[0] + 1

    # 2. Initialize an empty string variable for accumulating values from column A
    slot1_str = ""

    # 4. Iterate over the rows starting at start_row
    for idx in range(start_row, len(data)):
        # Get the value from column A (index 0) and the corresponding n from column C (index 2)
        cell_value = data.iloc[idx, 0]
        try:
            n = int(data.iloc[idx, 2])
        except (ValueError, TypeError):
            # If conversion to integer fails, skip this row
            continue

        # Only process if n is positive
        if n > 0:
            # Repeat the value n times, separated by commas
            repeated_value = ",".join([str(cell_value)] * n)
            # Append to slot1_str, adding a comma if slot1_str is not empty
            if slot1_str:
                slot1_str += "," + repeated_value
            else:
                slot1_str = repeated_value

    # Add the result to the DataFrame using .loc
    new_row = {"ID": file_id, "slot2": "AddPart:"+slot1_str, "Description": product_desc,"slot1":"MISC:1:1:P"}
    df.loc[len(df)] = new_row

# Optional: display or save the resulting DataFrame
df.to_csv('Promos.csv',index=False)

In [10]:
import pandas as pd
import re

def process_excel(file_path: str) -> pd.DataFrame:
    """
    Processes an Excel file sheet by sheet, producing one row per sheet whose name matches the pattern xxx-xxx:
    - ID: sheet name
    - Description: cell A1 with the sheet name stripped out
    - slot1: "MISC:1:1:P"
    - slot2: "AddPart:" + comma‑separated repeats of column A values, repeated by the count in column C
    Sheets not matching the pattern \d{3}-\d{3} are skipped.
    """
    id_pattern = re.compile(r'^\d{3}-\d{3}$')
    xls = pd.ExcelFile(file_path)
    df = pd.DataFrame(columns=["ID", "Description", "Notes", "slot1", "slot2"])

    for sheet_name in xls.sheet_names:
        # Skip sheets whose name doesn't match xxx-xxx
        if not id_pattern.match(sheet_name):
            continue

        file_id = sheet_name
        data = pd.read_excel(xls, sheet_name=sheet_name, header=None, dtype=str)

        # Description from A1, with sheet_name stripped
        raw_desc = data.iat[0, 0] or ""
        product_desc = raw_desc.replace(file_id, "").strip()

        # Build slot1_str by iterating from row 3 onward (index 2)
        slot1_str = ""
        start_row = 2
        for idx in range(start_row, len(data)):
            cell_value = data.iat[idx, 0]
            try:
                n = int(data.iat[idx, 2])
            except (ValueError, TypeError):
                continue

            if n > 0:
                repeated_value = ",".join([str(cell_value)] * n)
                if slot1_str:
                    slot1_str += "," + repeated_value
                else:
                    slot1_str = repeated_value

        # Add one row per valid sheet
        new_row = {
            "ID": file_id,
            "Description": product_desc,
            "Notes": "",
            "slot1": "MISC:1:1:P",
            "slot2": "AddPart:" + slot1_str
        }
        df.loc[len(df)] = new_row

    return df

path_to_excel = "2025 Systems Promo Breakouts_V2.xlsx"
result_df = process_excel(path_to_excel)
result_df.to_csv('Promos_product.csv',index=False)

In [3]:
import pdfplumber
import json
from pprint import pprint

PDF_PATH = "tmp.pdf"
OUTPUT_JSON = "quote_sections.json"

def extract_sections_with_headers(pdf_path):
    results = []

    with pdfplumber.open(pdf_path) as pdf:
        for pnum, page in enumerate(pdf.pages, start=1):
            print(f"\n=== PAGE {pnum} ===")

            # 1️⃣ Get tables (with bounding boxes)
            tables = page.find_tables()
            table_bounds = [(t.bbox[1], t.bbox[3]) for t in tables]  # (y0, y1)
            table_data = [t.extract() for t in tables]

            # 2️⃣ Get all text lines with positions
            words = page.extract_words(use_text_flow=True, keep_blank_chars=False)
            text_blocks = {}
            for w in words:
                y_center = round((w["top"] + w["bottom"]) / 2)
                text_blocks.setdefault(y_center, []).append(w["text"])
            lines = [" ".join(v) for k, v in sorted(text_blocks.items(), reverse=True)]

            print(f"Found {len(lines)} lines of text, {len(table_data)} tables")

            # 3️⃣ Identify section-like lines (between tables)
            section_candidates = []
            y_coords_sorted = sorted(text_blocks.keys(), reverse=True)

            for y in y_coords_sorted:
                # check if line y is outside any table
                in_table = any(low <= y <= high for (low, high) in table_bounds)
                if not in_table:
                    text_line = " ".join(text_blocks[y])
                    # section headers often contain words like microscope/system
                    if any(k in text_line.lower() for k in ["microscope", "system", "camera"]):
                        section_candidates.append((y, text_line))
                        print(f"Potential section header: {text_line}")

            # 4️⃣ Sort top→bottom and pair each header to next table
            section_candidates.sort(key=lambda x: x[0], reverse=True)
            for i, (y, header) in enumerate(section_candidates):
                next_table = None
                for t_idx, (t_low, t_high) in enumerate(table_bounds):
                    if t_high < y:
                        next_table = t_idx
                        break
                if next_table is not None:
                    items = table_data[next_table]
                    results.append({"header": header, "rows": items})
                else:
                    print(f"No table found below header: {header}")

    # 5️⃣ Clean and normalize
    structured = []
    for sec in results:
        header = sec["header"]
        table = sec["rows"]
        if not table or len(table[0]) < 4:
            continue
        items = []
        for r in table[1:]:  # skip header row
            if len(r) < 4: continue
            qty, product, desc, coo = r
            if not qty.strip(): continue
            items.append({
                "qty": qty.strip(),
                "product": product.strip(),
                "description": desc.strip(),
                "coo": coo.strip(),
            })
        structured.append({"header": header, "items": items})
    return structured


if __name__ == "__main__":
    data = extract_sections_with_headers(PDF_PATH)
    pprint(data[:2])
    print(f"\nTotal sections found: {len(data)}")

    with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    print(f"\n✅ Output saved to {OUTPUT_JSON}")



=== PAGE 1 ===
Found 44 lines of text, 1 tables

=== PAGE 2 ===
Found 51 lines of text, 1 tables

=== PAGE 3 ===
Found 50 lines of text, 1 tables

=== PAGE 4 ===
Found 52 lines of text, 0 tables
Potential section header: motorized microscope drivers. Optional modules include: EDF, HDR, AutoMeasure, Grain Sizing, Cast Iron
Potential section header: NIS-Elements Documentation (D) Package with single document interface. Includes camera and Nikon
Potential section header: C-FL Quad DAPI/FITC/TRITC/CY5 LED Filter Set for Nikon D-LEDI LED Fluorescence Illumination System
Potential section header: Wavelengths. Direct Couple to Microscope. Remote Control Pad Included. Requires MQF52057 AC Power
Potential section header: D-LEDI Fluorescence LED Illumination System. Four Channel Epi- Fluorescence LED illumination System
Potential section header: System.
No table found below header: motorized microscope drivers. Optional modules include: EDF, HDR, AutoMeasure, Grain Sizing, Cast Iron
No table fo